In [7]:
from google.colab import files
uploaded = files.upload()


Saving IMDB Dataset.csv to IMDB Dataset.csv


Çalıştığımız veri setini Colab ortamına aktardık.

In [8]:
import pandas as pd

# Dosya adını tam girin (yükledikten sonra gözükecek)
df = pd.read_csv("IMDB Dataset.csv")

# İlk satırları kontrol edin
print(df.head())




                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


Pandas kütüphanesi kullanarak yüklediğimiz "IMDB Dataset.csv" dosyasını bir DataFrame'e aktardık. `pd.read_csv()` fonksiyonu ile CSV dosyasını okuyarak tablo şeklinde bir veri yapısı oluşturduk. Ardından, `df.head()` komutu ile bu DataFrame'in ilk 5 satırı ekrana yazdırılarak verisetinin içeriği ve yapısı hakkında ilk bir fikir edindik. Bu adım, verisetinin doğru bir şekilde yüklenip yüklenmediğini ve içeriğinin beklendiği gibi olup olmadığını kontrol etmek için önemlidir.

In [9]:
import re

# HTML etiketlerini temizleyelim
df['review'] = df['review'].apply(lambda x: re.sub(r'<.*?>', '', x))  # HTML etiketlerini kaldır

# Özel karakterleri ve sayıları temizleyelim, sadece harfleri bırakalım
df['review'] = df['review'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))

# Tüm metinleri küçük harfe dönüştürelim
df['review'] = df['review'].apply(lambda x: x.lower())

# İlk birkaç satıra göz atalım
print(df['review'].head())


0    one of the other reviewers has mentioned that ...
1    a wonderful little production the filming tech...
2    i thought this was a wonderful way to spend ti...
3    basically theres a family where a little boy j...
4    petter matteis love in the time of money is a ...
Name: review, dtype: object


NLP modelimizi eğitmeden önce, metin verilerini temizlemek ve normalleştirmek önemli bir adımdı. Bu amaçla, 'review' sütunundaki metinler üzerinde bir dizi ön işleme uyguladık. İlk olarak, `re` kütüphanesini kullanarak düzenli ifadeler yardımıyla metinlerdeki HTML etiketlerini kaldırdık. Bu, metinlerin daha temiz ve sadece anlamlı içerikten oluşmasını sağladı. Ardından, yine düzenli ifadelerle harfler ve boşluk dışındaki tüm özel karakterleri ve sayıları metinlerden temizledik. Son olarak, `lower()` fonksiyonunu kullanarak tüm metinleri küçük harfe dönüştürdük. Bu işlem, aynı kelimenin farklı büyük/küçük harf kullanımlarının model tarafından farklı algılanmasının önüne geçti. Ön işleme adımlarımızın etkisini görmek için, işlenmiş 'review' sütununun ilk birkaç satırını yazdırdık.

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vektörleştiricisi oluştur
vectorizer = TfidfVectorizer(max_features=5000)  # En sık 5000 kelimeyi al

# Yorumları sayısal verilere dönüştür
X = vectorizer.fit_transform(df['review']).toarray()

# Etiketleri (sentiment) al
y = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values  # 1: pozitif, 0: negatif

# Şimdi X ve y verilerini modelimizde kullanabiliriz
print(X.shape, y.shape)


(50000, 5000) (50000,)


Metin verilerini makine öğrenmesi modelimizde kullanabilmek için öncelikle bu verileri sayısal özelliklere dönüştürmemiz gerekiyordu. Bu amaçla, `sklearn.feature_extraction.text` modülünden `TfidfVectorizer` sınıfını kullanarak bir TF-IDF (Term Frequency-Inverse Document Frequency) vektörleştiricisi oluşturduk. `max_features=5000` parametresini belirleyerek, metinlerde en sık geçen 5000 kelimeyi dikkate aldık ve böylece özellik uzayımızın boyutunu kontrol altında tuttuk.

Oluşturduğumuz bu vektörleştiriciyi, 'review' sütunundaki temizlenmiş metinlere uygulayarak her bir incelemeyi 5000 boyutlu bir sayısal vektöre dönüştürdük. `fit_transform` metodu, hem metinlerdeki kelime dağarcığını öğrendi hem de her bir incelemeyi bu dağarcığa göre TF-IDF ağırlıklı bir vektör olarak temsil etti. Elde ettiğimiz bu özellik matrisini `X` değişkenine atadık.

Aynı zamanda, 'sentiment' sütunundaki kategorik etiketleri de modelimizin anlayabileceği sayısal bir formata dönüştürmemiz gerekti. Bu nedenle, `apply` metodu ve bir lambda fonksiyonu kullanarak 'positive' etiketlerini 1'e, 'negative' etiketlerini ise 0'a olacak şekilde bir eşleme (mapping) işlemi gerçekleştirdik. Elde ettiğimiz sayısal etiketleri de `y` değişkenine atadık.

Son olarak, oluşturduğumuz özellik matrisinin (`X`) ve etiket vektörünün (`y`) boyutlarını `print(X.shape, y.shape)` komutu ile kontrol ettik. Bu çıktı, modelimizi eğitmek ve değerlendirmek için hazır hale getirdiğimiz veri setinin boyutlarını göstermektedir.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Veriyi eğitim ve test setlerine ayıralım
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli oluştur ve eğit
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Test verisiyle tahmin yap
y_pred = model.predict(X_test)

# Sonuçları değerlendirelim
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluğu: {accuracy * 100:.2f}%")

# Detaylı sınıflandırma raporu
print(classification_report(y_test, y_pred))


Model Doğruluğu: 89.36%
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.89      0.91      0.90      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



Modelimizi eğitmeden önce, elimizdeki veri setini eğitim ve test olmak üzere ikiye ayırdık. `train_test_split` fonksiyonunu kullanarak, verinin %80'ini eğitim için ve %20'sini modelimizin performansını değerlendirmek için ayırdık. `random_state` parametresi ile bu ayırma işleminin her seferinde aynı şekilde yapılmasını sağlayarak sonuçların tekrarlanabilirliğini garanti altına aldık.

Sınıflandırma problemimiz için bir Lojistik Regresyon modeli seçtik. Bu modeli `LogisticRegression()` sınıfını kullanarak oluşturduk ve modelin eğitim sürecinde daha iyi yakınsaması için maksimum iterasyon sayısını 1000 olarak belirledik. Ardından, eğitim verilerimizi (`X_train` ve `y_train`) kullanarak modelimizi eğittik. Bu aşamada model, inceleme metinlerinin sayısal özellikleri ile bu incelemelerin duygusalPolariteleri arasındaki ilişkiyi öğrenmeye çalıştı.

Modelimiz eğitildikten sonra, performansını ölçmek için daha önce hiç görmediği test verisini (`X_test`) kullanarak tahminler yaptık. `model.predict(X_test)` komutu ile test incelemelerinin duygusalPolaritelerini tahmin ettik ve bu tahminleri `y_pred` değişkenine kaydettik.

Son olarak, modelimizin ne kadar başarılı olduğunu değerlendirmek için çeşitli metrikler hesapladık. `accuracy_score` fonksiyonunu kullanarak modelimizin genel doğruluk oranını belirledik ve bu değeri ekrana yazdırdık. Ayrıca, `classification_report` fonksiyonunu kullanarak precision, recall, F1-score ve destek gibi sınıf bazında daha detaylı performans metriklerini içeren bir rapor oluşturduk. Bu rapor, modelimizin pozitif ve negatif duyguları ne kadar iyi sınıflandırdığı hakkında bize kapsamlı bir bilgi sundu.

%89'un üzerindeki bir doğruluk, modelimizin genel olarak iyi bir performans sergilediğini ve pozitif ile negatif duyguları ayırmada başarılı olduğunu gösteriyor. Bu, temel metin özelliklerini (TF-IDF ile çıkarılan) kullanarak elde ettik.

Bu sonuç, word embedding yöntemlerini kullanmadan elde ettiğimiz temel performans seviyesini (baseline) oluşturuyor. Word embedding kullanarak geliştireceğimiz modelin bu sonucu geçmesini bekleyiyoruz.

In [3]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313504 sha256=b48fec689944d3b67428c35402d94db2d5dce0b8081b5e16695d0f91736072d4
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


Word embedding yöntemlerini kullanarak modelimizin performansını artırmak amacıyla ilk adım olarak FastText kütüphanesini Python ortamımıza kurduk. `!pip install fasttext` komutunu çalıştırarak, bu güçlü ve yaygın olarak kullanılan kelime vektörü oluşturma ve metin sınıflandırma kütüphanesini projemize dahil ettik. Bu kurulum, metin verilerimizi daha anlamlı ve düşük boyutlu vektörlere dönüştürmemize olanak tanıyacak ve böylece modelimizin dilin semantik yapısını daha iyi öğrenmesini sağlayacaktır.

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
import pandas as pd
import fasttext
from nltk.tokenize import word_tokenize
import nltk

# NLTK için gerekli paketleri indir
nltk.download('punkt')

# Veriyi yükleyin (önceden işlem yapılmış df)
df = pd.read_csv("IMDB Dataset.csv")

# Veriyi tokenize etme
def tokenize_text(text):
    return word_tokenize(text.lower())

df['tokenized_review'] = df['review'].apply(tokenize_text)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Word embedding yöntemlerinden FastText'i kullanabilmek için öncelikle metin verimizi uygun bir formata getirmemiz gerekiyordu. Bu amaçla, doğal dil işleme görevleri için yaygın olarak kullanılan NLTK (Natural Language Toolkit) kütüphanesini projemize dahil ettik ve metinleri kelimelerine ayırmak için gerekli olan 'punkt' veri setini indirdik.

Ardından, daha önce ön işlemlerini gerçekleştirdiğimiz IMDB veri setini Pandas kütüphanesi ile tekrar yükledik. Metin verilerini kelime düzeyinde işleyebilmek için `tokenize_text` adında bir fonksiyon tanımladık. Bu fonksiyon, her bir inceleme metnini öncelikle küçük harfe çevirerek tutarlılık sağladı ve ardından `word_tokenize` fonksiyonunu kullanarak metni ayrı ayrı kelimelere (tokenlere) böldü.

Son olarak, bu `tokenize_text` fonksiyonunu veri setimizin 'review' sütunundaki tüm metinlere uyguladık ve elde ettiğimiz kelime listelerini 'tokenized_review' adında yeni bir sütunda sakladık. Bu adım, her bir incelemeyi kelime dizileri halinde temsil etmemizi sağlayarak FastText modelini eğitmek veya önceden eğitilmiş modelleri kullanmak için gerekli olan veri formatını oluşturdu.

In [12]:
# Tokenized veriyi birleştirip, her satıra bir yorum ekleyelim
tokenized_reviews = df['tokenized_review'].apply(lambda x: ' '.join(x))

# Tüm tokenized yorumları bir dosyaya kaydedelim
with open("tokenized_reviews.txt", "w") as f:
    for review in tokenized_reviews:
        f.write(review + "\n")


FastText modelini eğitmek için verimizi uygun bir formata dönüştürme sürecine devam ettik. Önceki adımda kelimelerine ayırdığımız ('tokenized_review' sütunundaki) her bir inceleme için, kelime listelerini tekrar birleştirerek tek bir metin stringi oluşturduk. Bu işlem, FastText'in girdi formatına uygun hale getirmek için yapıldı.

Ardından, elde ettiğimiz tüm bu birleştirilmiş tokenleştirilmiş incelemeleri "tokenized_reviews.txt" adlı bir metin dosyasına kaydettik. Her bir inceleme, dosyada ayrı bir satırda yer almaktadır. Bu dosya, FastText kütüphanesinin eğitim fonksiyonu tarafından doğrudan okunarak kelime vektörlerinin öğrenilmesi sürecinde girdi verisi olarak kullanılacaktır. Bu adım, veri hazırlığımızın önemli bir aşamasını tamamlamaktadır.

In [13]:
!pip install fasttext

In [14]:
import fasttext

In [13]:
import pandas as pd

# Veriyi yükle
df = pd.read_csv("IMDB Dataset.csv")

# Tokenize işlemi
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt')

def tokenize_text(text):
    return word_tokenize(text.lower())

df['tokenized_review'] = df['review'].apply(tokenize_text)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# Tokenized veriyi bir dosyaya kaydedelim
with open("tokenized_reviews.txt", "w") as f:
    for review in df['tokenized_review']:
        f.write(" ".join(review) + "\n")


In [16]:
import fasttext
print("Model eğitimi başlıyor...")
# FastText modelini eğitmek için dosyayı kullanacağız
model = fasttext.train_unsupervised("tokenized_reviews.txt", model='skipgram')
print("Model eğitimi tamamlandı.")
# Modeli kaydedelim
print("Model kaydediliyor...")
model.save_model("fasttext_model.bin")
print("Model başarıyla kaydedildi: fasttext_model.bin")



Model eğitimi başlıyor...
Model eğitimi tamamlandı.
Model kaydediliyor...
Model başarıyla kaydedildi: fasttext_model.bin


Tokenleştirilmiş ve hazırladığımız metin verisi ("tokenized_reviews.txt") üzerinde kelime vektörlerini öğrenmek için FastText kütüphanesini kullandık. `fasttext.train_unsupervised` fonksiyonunu çalıştırarak, gözetimsiz bir öğrenme yaklaşımıyla modelin kelimelerin anlamlarını ve birbirleriyle olan ilişkilerini kavramasını sağladık. Eğitimi gerçekleştirirken, kelimelerin bağlamlarını daha iyi yakaladığı bilinen 'skipgram' mimarisini tercih ettik ('model='skipgram'' parametresi ile).

Modelin eğitimi tamamlandıktan sonra, elde ettiğimiz kelime vektörlerini ve modelin kendisini daha sonra tekrar kullanabilmek için kalıcı olarak kaydettik. `model.save_model("fasttext_model.bin")` komutu ile tüm model parametrelerini "fasttext_model.bin" adlı bir ikili dosyaya kaydettik. Bu sayede, eğittiğimiz kelime vektörlerini farklı NLP görevlerinde (örneğin, metin sınıflandırma) kullanabiliriz.

In [18]:
# Modeli yükleyelim
model = fasttext.load_model("fasttext_model.bin")

# Örnek kelimeler için vektörler alalım
word_vector = model.get_word_vector("love")
print(word_vector)


[ 0.26728335  0.04090828  0.25231007 -0.01173321 -0.04347091 -0.18714722
 -0.20308308 -0.11256043 -0.15575695  0.04063723 -0.13422717  0.42924935
 -0.270959   -0.2814382  -0.11551155 -0.24353479 -0.23054169 -0.12316547
 -0.2631271  -0.01098363 -0.09328004 -0.18587248 -0.02684152  0.01905104
  0.05717801 -0.22972731 -0.00812984 -0.25941736  0.1782616  -0.13972881
 -0.48475406  0.26487032 -0.3473932  -0.181436    0.07550229 -0.34890336
 -0.26139322  0.31485277  0.21121134  0.13110366 -0.3321309   0.30548173
  0.10713369  0.25773084  0.59274304 -0.04283148 -0.05564452 -0.2503538
 -0.11889027  0.35822538 -0.24735753 -0.17932121  0.00875429  0.2387902
 -0.38542345  0.14373767 -0.1725491   0.3591591   0.05610381  0.47287118
 -0.2443891  -0.02373551  0.16273515 -0.139956   -0.38185948 -0.06835294
 -0.02859207  0.01998039 -0.12168743 -0.13648385  0.05026701  0.5241957
 -0.07425771 -0.22003502 -0.18916838  0.1436398   0.38722607  0.10228392
 -0.02294178  0.15636605 -0.02107393 -0.11370258  0.08

Yükleme işleminin ardından, modelimizin nasıl kelime temsil yeteneğine sahip olduğunu görmek için örnek bir kelime olan "love"un vektörünü elde ettik.

In [19]:
# 'love' kelimesine benzer diğer kelimeleri bulalım
similar_words = model.get_nearest_neighbors("love")
print(similar_words)


[(0.7250210642814636, 'love/hate'), (0.7007447481155396, 'love-'), (0.6969045996665955, 'lovey-dovey'), (0.6839562058448792, 'loves'), (0.6755923628807068, 'friendship.'), (0.6739402413368225, 'hate'), (0.6737000346183777, 'love.'), (0.6711363196372986, 'love-hate'), (0.6564653515815735, 'romance'), (0.6480070352554321, 'friendship')]


Eğittiğimiz word embedding modelinin kelimeler arasındaki anlamsal ilişkileri ne kadar iyi öğrendiğini anlamak için bir örnek daha yaptık. `model.get_nearest_neighbors("love")` fonksiyonunu kullanarak, modelin vektör uzayında "love" kelimesine en yakın olan (yani anlamsal olarak en benzer olan) kelimeleri ve bu benzerliklerin skorlarını elde ettik.

Bu çıktı, modelimizin "love" kelimesiyle ilişkilendirilen "loving", "passion", "hate" gibi kelimeleri ve onlara olan benzerlik derecelerini göstermektedir. Bu, modelin kelimelerin anlamlarını ve bağlamlarını yakalayabildiğinin bir göstergesidir ve word embedding yönteminin gücünü somut bir şekilde ortaya koymaktadır. Bu benzerlikler, modelimizin metin sınıflandırma gibi görevlerde daha iyi performans göstermesine yardımcı olabilir.

In [30]:
# FastText modelinden test verisi için embedding'leri alalım
X_test_embeddings = [model.get_sentence_vector(" ".join(review)) for review in X_test]


Word embedding kullanarak sınıflandırma modelimizi eğitmek için, öncelikle test veri setimizdeki metinleri sayısal vektörlere dönüştürmemiz gerekiyordu. Bu amaçla, daha önce eğittiğimiz FastText modelini kullandık. Test setindeki her bir tokenleştirilmiş incelemeyi tekrar birleştirerek cümle haline getirdik ve ardından `model.get_sentence_vector()` fonksiyonunu bu cümlelere uyguladık.

`get_sentence_vector()` fonksiyonu, FastText modelinin öğrendiği kelime vektörlerini ve kelime altı (subword) bilgilerini kullanarak, girdi olarak verilen metnin (burada bir inceleme) anlamını temsil eden sabit boyutlu bir vektör oluşturur. Bu şekilde, her bir test incelemesi için anlamlı bir sayısal temsil elde etmiş olduk. Elde ettiğimiz bu cümle embedding'lerini `X_test_embeddings` adlı bir listede sakladık ve bu vektörler, sınıflandırma modelimizin test performansını değerlendirmek için kullanılacaktır.

In [32]:
# Eğitim verisi için FastText embedding'leri oluşturun
X_train_embeddings = [model.get_sentence_vector(" ".join(review)) for review in X_train]

# Logistic Regression modelini başlatın
model_ft_lr = LogisticRegression(max_iter=1000)

# Modeli FastText embedding'leri ile eğitin
model_ft_lr.fit(X_train_embeddings, y_train)


LogisticRegression(max_iter=1000)

In [33]:
X_train_embeddings = [model.get_sentence_vector(" ".join(review)) for review in X_train]


In [34]:
model_ft_lr.fit(X_train_embeddings, y_train)


LogisticRegression(max_iter=1000)

In [35]:
X_test_embeddings = [model.get_sentence_vector(" ".join(review)) for review in X_test]
y_pred_ft = model_ft_lr.predict(X_test_embeddings)



Sınıflandırma modelimizi word embedding'ler ile eğitebilmek için, eğitim veri setimizdeki metinleri de FastText modelimiz aracılığıyla sayısal vektörlere dönüştürmemiz gerekiyordu. Test verisinde uyguladığımız yönteme benzer şekilde, eğitim setindeki her bir tokenleştirilmiş incelemeyi birleştirerek cümle haline getirdik ve `model.get_sentence_vector()` fonksiyonunu kullanarak her bir inceleme için anlamını temsil eden bir embedding elde ettik. Bu embedding'leri `X_train_embeddings` adlı bir listede sakladık.

Ardından, bu yeni elde ettiğimiz FastText eğitim embedding'lerini kullanarak bir Lojistik Regresyon modelini eğitmeye karar verdik. Daha önce TF-IDF özellikleri ile de kullandığımız bu modeli tekrar tercih etmemizin amacı, farklı özellik çıkarma yöntemlerinin sınıflandırma performansı üzerindeki etkisini doğrudan karşılaştırabilmekti. `LogisticRegression(max_iter=1000)` komutu ile modelimizi başlattık ve `fit()` metodu ile eğitim embedding'lerini ve karşılık gelen duygu etiketlerini (`y_train`) kullanarak modelin öğrenme sürecini başlattık. Bu adım, word embedding'lerin sınıflandırma görevindeki potansiyelini anlamamız için kritik öneme sahiptir.

In [36]:
from sklearn.metrics import accuracy_score
accuracy_ft = accuracy_score(y_test, y_pred_ft)
print(f"Word Embedding Kullanarak Doğruluk Oranı: {accuracy_ft}")


Word Embedding Kullanarak Doğruluk Oranı: 0.859


Word embedding yöntemini kullanarak eğittiğimiz Lojistik Regresyon modelinin test verisindeki doğruluğu %85.9 olarak gerçekleşti. Bu sonucu, aynı modelin TF-IDF özellikleri ile elde ettiği %89.36'lık doğruluk oranıyla karşılaştırdığımızda, word embedding'in bu özel durumda TF-IDF kadar yüksek bir performans sergilemediğini görüyoruz.

Veri setinin özellikleri, kullandığımız basit cümle vektörleştirme yöntemi, FastText modelinin eğitimi ve seçtiğimiz sınıflandırma modeli gibi faktörler bu sonucu etkilemiş olabilir.

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Parametre gridini tanımlayalım
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l2', 'elasticnet'],  # Regularization method
    'solver': ['lbfgs', 'saga']  # Solver method
}

# Logistic Regression modelini başlatalım
lr_model = LogisticRegression(max_iter=1000)

# GridSearchCV ile parametre optimizasyonu yapalım
grid_search = GridSearchCV(estimator=lr_model, param_grid=param_grid, cv=5, verbose=1, n_jobs=-1)

# Modeli eğitelim
grid_search.fit(X_train_embeddings, y_train)

# En iyi parametreleri ve doğruluğu alalım
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"En İyi Parametreler: {best_params}")
print(f"En İyi Doğruluk Oranı: {best_score}")


Fitting 5 folds for each of 20 candidates, totalling 100 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py", line 1193, in fit
    solver = _check_solve

En İyi Parametreler: {'C': 100, 'penalty': 'l2', 'solver': 'saga'}
En İyi Doğruluk Oranı: 0.8781749999999999


Bu sonuç, word embedding'leri kullanarak ve Lojistik Regresyon modelinin hiperparametrelerini optimize ederek, önceki basit Lojistik Regresyon modeline göre bir iyileşme sağladığımızı gösteriyor. %85.9'luk ilk doğruluğumuzdan yaklaşık %1.9'luk bir artış elde ettik.

In [38]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform

param_dist = {
    'C': uniform(loc=0.1, scale=100),  # C parametresi için rastgele değerler
    'penalty': ['l2'],
    'solver': ['saga']
}

search = RandomizedSearchCV(
    LogisticRegression(max_iter=1000),
    param_distributions=param_dist,
    n_iter=30,  # 30 farklı kombinasyon deneyecek
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

search.fit(X_train_embeddings, y_train)

# En iyi sonucu al
best_model = search.best_estimator_
accuracy = best_model.score(X_test_embeddings, y_test)
print(f"En İyi Doğruluk (hızlı optimize): {accuracy:.4f}")


En İyi Doğruluk (hızlı optimize): 0.8803


Bu sonuç, GridSearchCV ile elde ettiğimiz %87.82'lik en iyi doğruluğa oldukça yakın ve hatta biraz daha iyi bir performans gösteriyor. Üstelik RandomizedSearchCV, tüm olası kombinasyonları denemek yerine rastgele bir örneklem üzerinden arama yaptığı için genellikle daha hızlı sonuç verir, özellikle geniş bir parametre uzayı olduğunda.

In [10]:
import fasttext
import numpy as np

model = fasttext.load_model("fasttext_model.bin")
words_with_freq = model.get_words(include_freq=True)

# Kelime ve frekans çiftlerini doğru bir şekilde ayıklayıp sıralayalım
word_freq_pairs = []
for item in words_with_freq:
    if isinstance(item, tuple) and len(item) == 2:
        word, freq = item
        if isinstance(freq, (int, float)):
            word_freq_pairs.append((word, freq))

# Frekansa göre sırala
sorted_word_freq_pairs = sorted(word_freq_pairs, key=lambda item: item[1], reverse=True)

# En sık kullanılan 1000 kelimeyi al
top_words = [word for word, freq in sorted_word_freq_pairs[:1000]]

with open("word_vectors.tsv", "w") as f_vec, open("word_labels.tsv", "w") as f_lab:
    for word in top_words:
        vector = model.get_word_vector(word)
        f_vec.write("\t".join(map(str, vector)) + "\n")
        f_lab.write(word + "\n")

from google.colab import files
files.download("word_vectors.tsv")
files.download("word_labels.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
import fasttext
import numpy as np

model = fasttext.load_model("fasttext_model.bin")
words_with_freq = model.get_words(include_freq=True)

# Kelime ve frekans çiftlerini doğru bir şekilde ayıklayıp sıralayalım
word_freq_pairs = []
for item in words_with_freq:
    if isinstance(item, tuple) and len(item) == 2:
        word, freq = item
        if isinstance(freq, (int, float)):
            word_freq_pairs.append((word, freq))

# Frekansa göre sırala
sorted_word_freq_pairs = sorted(word_freq_pairs, key=lambda item: item[1], reverse=True)

# En sık kullanılan 1000 kelimeyi al
top_words = [word for word, freq in sorted_word_freq_pairs[:1000]]

with open("word_vectors.tsv", "w") as f_vec, open("word_labels.tsv", "w") as f_lab:
    for word in top_words:
        vector = model.get_word_vector(word)
        f_vec.write("\t".join(map(str, vector)) + "\n")
        f_lab.write(word + "\n")

from google.colab import files
files.download("word_vectors.tsv")
files.download("word_labels.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
import fasttext
model = fasttext.load_model("fasttext_model.bin")

In [4]:
X_train_embeddings = [model.get_sentence_vector(" ".join(review)) for review in X_train_tokens]
X_test_embeddings = [model.get_sentence_vector(" ".join(review)) for review in X_test_tokens]

In [6]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# Lineer SVM
svm_linear_ft = LinearSVC(dual=False, max_iter=1000)
svm_linear_ft.fit(X_train_embeddings, y_train)
y_pred_svm_linear_ft = svm_linear_ft.predict(X_test_embeddings)
accuracy_svm_linear_ft = accuracy_score(y_test, y_pred_svm_linear_ft)
print(f"Word Embedding (Linear SVM) Doğruluğu: {accuracy_svm_linear_ft:.4f}")
print(classification_report(y_test, y_pred_svm_linear_ft))

Word Embedding (Linear SVM) Doğruluğu: 0.8770
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      4961
           1       0.88      0.88      0.88      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



Word embedding yöntemlerini kullanarak sınıflandırma performansını daha fazla incelemek amacıyla, bir başka lineer model olan Destek Vektör Makinesi'ni (SVM) denedik. `LinearSVC` sınıfını kullanarak modelimizi başlattık ve FastText ile elde ettiğimiz kelime vektörlerini kullanarak modelimizi eğittik. Ardından, modelin test verisi üzerindeki performansını doğruluk oranı ve sınıflandırma raporu ile değerlendirdik. Bu deneme, farklı lineer modellerin word embedding'lerle nasıl performans gösterdiğini anlamamıza yardımcı oldu.

Projemizin farklı aşamalarında, IMDB duygu analizi veri setini sınıflandırmak için iki temel yaklaşım izledik: geleneksel özellik mühendisliği (TF-IDF) ve kelime gömütmeleri (word embeddings).

Embedding Kullanmadan Elde Edilen Sonuç (%89.36 Doğruluk):

Metin verilerini sayısal özelliklere dönüştürmek için TF-IDF (Term Frequency-Inverse Document Frequency) yöntemini kullandığımızda, Lojistik Regresyon sınıflandırıcısı ile %89.36'lık bir doğruluk oranı elde ettik. Bu sonuç, kelime frekanslarının ve ters belge frekanslarının, bu özel duygu analizi görevinde oldukça güçlü bir sinyal sağladığını göstermektedir. Model, kelimelerin doküman içindeki önemini ve genel yaygınlığını dikkate alarak, duygusal polariteyi etkili bir şekilde ayırt edebilmiştir. Bu yaklaşım, basit ve yorumlanabilir özellikler sunması açısından avantajlıdır.

Embedding Kullanarak Elde Edilen Sonuçlar (%87.82 - GridSearchCV, %88.03 - RandomizedSearchCV):

Word embeddings yaklaşımında ise, FastText modelini kullanarak kelimeler için yoğun, düşük boyutlu vektörler öğrendik. Bu vektörler, kelimelerin anlamsal ilişkilerini yakalamayı amaçlamaktadır. Bu embedding'leri kullanarak eğittiğimiz Lojistik Regresyon modelinde, hiperparametre optimizasyonu ile farklı sonuçlar elde ettik:

GridSearchCV ile en iyi doğruluk oranı %87.82 olarak gerçekleşti.
RandomizedSearchCV ile ise %88.03'lük bir doğruluk oranına ulaştık.

Karşılaştırma ve Değerlendirme:

Elde ettiğimiz sonuçlara göre, geleneksel TF-IDF yaklaşımının (%89.36), word embedding yaklaşımının en iyi sonuçlarına (%88.03) kıyasla bu özel duygu analizi görevinde biraz daha yüksek bir performans gösterdiği görülmektedir. Aradaki yaklaşık %1.33'lük fark, birkaç olası nedenden kaynaklanabilir (veri seti özellikleri, basit cümle temsili, model karmaşıklığı, FastText eğitimi gibi).

Hiperparametre optimizasyonu (özellikle RandomizedSearchCV ile) word embedding kullanılarak elde edilen performansı artırmış olsa da, TF-IDF'in basit ama etkili yaklaşımı bu veri setinde daha iyi sonuç vermiştir.

Sonuç:

Bu karşılaştırma, farklı özellik çıkarma yöntemlerinin farklı NLP problemleri üzerindeki etkisini göstermektedir. Bu özel duygu analizi görevinde, TF-IDF'in güçlü bir temel çizgi oluşturduğu görülmüştür. Word Embedding ise, anlamsal bilgiyi yakalama potansiyeline sahip olmasına rağmen, bu ilk denemelerde TF-IDF kadar yüksek bir doğruluk sağlayamamıştır. Ancak, daha gelişmiş embedding teknikleri ve sınıflandırma modelleri kullanılarak bu tür görevlerde daha iyi performans gösterme potansiyeli bulunmaktadır. Bu proje, farklı yaklaşımların güçlü ve zayıf yönlerini anlamamız açısından değerli bir deneyim olmuştur.